### DOWNLOADING THE DATASET

In [1]:
from keras.datasets import cifar10
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()


Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

### PREPROCESSING THE DATA

In [3]:
y_train_cat=to_categorical(y_train)
y_test_cat=to_categorical(y_test)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train_cat, batch_size=30)
val_generator = val_datagen.flow(X_test, y_test_cat, batch_size=20)

In [5]:
input_shape=X_train[0].shape

### DEFINING THE MODEL

In [6]:

from keras.applications import vgg16
from keras.models import Model
import keras

vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

58892288/58889256 [==============================] - 74s 1us/step


,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x000001A351820D88>,input_1,False
1,<keras.layers.convolutional.Conv2D object at 0x000001A330CDD7C8>,block1_conv1,False
2,<keras.layers.convolutional.Conv2D object at 0x000001A3484F9708>,block1_conv2,False
3,<keras.layers.pooling.MaxPooling2D object at 0x000001A351856A88>,block1_pool,False
4,<keras.layers.convolutional.Conv2D object at 0x000001A3517CDA08>,block2_conv1,False
5,<keras.layers.convolutional.Conv2D object at 0x000001A3517D7508>,block2_conv2,False
6,<keras.layers.pooling.MaxPooling2D object at 0x000001A3517D9988>,block2_pool,False
7,<keras.layers.convolutional.Conv2D object at 0x000001A3517DDC88>,block3_conv1,False
8,<keras.layers.convolutional.Conv2D object at 0x000001A3517E1E48>,block3_conv2,False
9,<keras.layers.convolutional.Conv2D object at 0x000001A3517E47C8>,block3_conv3,False


### TRAINING THE MODEL

In [10]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
              
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=5
                              
                              
                              0,
                              validation_data=val_generator, validation_steps=50, 
                              verbose=1)              

Epoch 1/50
100/100 [==============================] - 28s 278ms/step - loss: 0.5211 - accuracy: 0.7821 - val_loss: 0.3701 - val_accuracy: 0.9000
Epoch 2/50
100/100 [==============================] - 25s 253ms/step - loss: 0.3526 - accuracy: 0.8994 - val_loss: 0.3190 - val_accuracy: 0.9000
Epoch 3/50
100/100 [==============================] - 27s 265ms/step - loss: 0.3332 - accuracy: 0.9000 - val_loss: 0.3205 - val_accuracy: 0.9000
Epoch 4/50
100/100 [==============================] - 27s 271ms/step - loss: 0.3294 - accuracy: 0.9000 - val_loss: 0.3085 - val_accuracy: 0.9000
Epoch 5/50
100/100 [==============================] - 27s 273ms/step - loss: 0.3265 - accuracy: 0.8999 - val_loss: 0.3006 - val_accuracy: 0.9000
Epoch 6/50
100/100 [==============================] - 27s 270ms/step - loss: 0.3226 - accuracy: 0.9000 - val_loss: 0.3077 - val_accuracy: 0.9000
Epoch 7/50
100/100 [==============================] - 26s 258ms/step - loss: 0.3198 - accuracy: 0.9000 - val_loss: 0.3021 - val_ac

In [23]:
model.save("cifar_model.h5")